# The end result of this exercise should be a file named acquire.py.

## Getting to Know API

In [2]:
import requests

base_url = 'https://python.zach.lol/documentation'

In [17]:
response = requests.get(base_url)
print(response.json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



In [18]:
response = requests.get('https://python.zach.lol/api/v1/items')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [19]:
data['payload'].keys()

dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [20]:
data['payload']['items'][:2]

[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'}]

### 1. Using the code from the lesson as a guide, create a dataframe named items that has all of the data for items.

In [22]:
import pandas as pd

In [23]:
df_items = pd.DataFrame(data['payload']['items'])
df_items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


### 2. Do the same thing, but for stores.

In [25]:
response = requests.get('https://python.zach.lol/api/v1/stores')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [27]:
df_stores = pd.DataFrame(data['payload']['stores'])
df_stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


### 3. Extract the data for sales. There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [30]:
response = requests.get('https://python.zach.lol/api/v1/sales')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [32]:
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

max_page: 183
next_page: /api/v1/sales?page=2


In [9]:
# in progress, still needs work

base_url = 'https://python.zach.lol/documentation'

response = requests.get(f"https://python.zach.lol/api/v1/sales?page={i}")
output = response.json()

for x in range(2, 184):
    
    response = requests.get(f"https://python.zach.lol/api/v1/sales?page={i}")
    data = response.json()
    output.append(data['payload']['sales'])
    
df_sales = pd.DataFrame(output)

NameError: name 'i' is not defined

In [52]:
df_sales

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,"{'item': 1, 'sale_amount': 13.0, 'sale_date': ...","{'item': 1, 'sale_amount': 11.0, 'sale_date': ...","{'item': 1, 'sale_amount': 14.0, 'sale_date': ...","{'item': 1, 'sale_amount': 13.0, 'sale_date': ...","{'item': 1, 'sale_amount': 10.0, 'sale_date': ...","{'item': 1, 'sale_amount': 12.0, 'sale_date': ...","{'item': 1, 'sale_amount': 10.0, 'sale_date': ...","{'item': 1, 'sale_amount': 9.0, 'sale_date': '...","{'item': 1, 'sale_amount': 12.0, 'sale_date': ...","{'item': 1, 'sale_amount': 9.0, 'sale_date': '...",...,"{'item': 1, 'sale_amount': 31.0, 'sale_date': ...","{'item': 1, 'sale_amount': 28.0, 'sale_date': ...","{'item': 1, 'sale_amount': 27.0, 'sale_date': ...","{'item': 1, 'sale_amount': 37.0, 'sale_date': ...","{'item': 1, 'sale_amount': 38.0, 'sale_date': ...","{'item': 1, 'sale_amount': 23.0, 'sale_date': ...","{'item': 1, 'sale_amount': 22.0, 'sale_date': ...","{'item': 1, 'sale_amount': 21.0, 'sale_date': ...","{'item': 1, 'sale_amount': 22.0, 'sale_date': ...","{'item': 1, 'sale_amount': 24.0, 'sale_date': ..."
1,"{'item': 1, 'sale_amount': 33.0, 'sale_date': ...","{'item': 1, 'sale_amount': 27.0, 'sale_date': ...","{'item': 1, 'sale_amount': 26.0, 'sale_date': ...","{'item': 1, 'sale_amount': 22.0, 'sale_date': ...","{'item': 1, 'sale_amount': 25.0, 'sale_date': ...","{'item': 1, 'sale_amount': 22.0, 'sale_date': ...","{'item': 1, 'sale_amount': 35.0, 'sale_date': ...","{'item': 1, 'sale_amount': 30.0, 'sale_date': ...","{'item': 1, 'sale_amount': 30.0, 'sale_date': ...","{'item': 1, 'sale_amount': 15.0, 'sale_date': ...",...,"{'item': 1, 'sale_amount': 10.0, 'sale_date': ...","{'item': 1, 'sale_amount': 20.0, 'sale_date': ...","{'item': 1, 'sale_amount': 12.0, 'sale_date': ...","{'item': 1, 'sale_amount': 19.0, 'sale_date': ...","{'item': 1, 'sale_amount': 18.0, 'sale_date': ...","{'item': 1, 'sale_amount': 15.0, 'sale_date': ...","{'item': 1, 'sale_amount': 24.0, 'sale_date': ...","{'item': 1, 'sale_amount': 13.0, 'sale_date': ...","{'item': 1, 'sale_amount': 17.0, 'sale_date': ...","{'item': 1, 'sale_amount': 20.0, 'sale_date': ..."
2,"{'item': 1, 'sale_amount': 15.0, 'sale_date': ...","{'item': 1, 'sale_amount': 21.0, 'sale_date': ...","{'item': 1, 'sale_amount': 17.0, 'sale_date': ...","{'item': 1, 'sale_amount': 20.0, 'sale_date': ...","{'item': 1, 'sale_amount': 12.0, 'sale_date': ...","{'item': 1, 'sale_amount': 24.0, 'sale_date': ...","{'item': 1, 'sale_amount': 17.0, 'sale_date': ...","{'item': 1, 'sale_amount': 18.0, 'sale_date': ...","{'item': 1, 'sale_amount': 23.0, 'sale_date': ...","{'item': 1, 'sale_amount': 24.0, 'sale_date': ...",...,"{'item': 1, 'sale_amount': 19.0, 'sale_date': ...","{'item': 1, 'sale_amount': 17.0, 'sale_date': ...","{'item': 1, 'sale_amount': 10.0, 'sale_date': ...","{'item': 1, 'sale_amount': 8.0, 'sale_date': '...","{'item': 1, 'sale_amount': 15.0, 'sale_date': ...","{'item': 1, 'sale_amount': 6.0, 'sale_date': '...","{'item': 1, 'sale_amount': 14.0, 'sale_date': ...","{'item': 1, 'sale_amount': 23.0, 'sale_date': ...","{'item': 1, 'sale_amount': 18.0, 'sale_date': ...","{'item': 1, 'sale_amount': 12.0, 'sale_date': ..."
3,"{'item': 1, 'sale_amount': 13.0, 'sale_date': ...","{'item': 1, 'sale_amount': 16.0, 'sale_date': ...","{'item': 1, 'sale_amount': 15.0, 'sale_date': ...","{'item': 1, 'sale_amount': 20.0, 'sale_date': ...","{'item': 1, 'sale_amount': 18.0, 'sale_date': ...","{'item': 1, 'sale_amount': 17.0, 'sale_date': ...","{'item': 1, 'sale_amount': 7.0, 'sale_date': '...","{'item': 1, 'sale_amount': 14.0, 'sale_date': ...","{'item': 1, 'sale_amount': 16.0, 'sale_date': ...","{'item': 1, 'sale_amount': 15.0, 'sale_date': ...",...,"{'item': 2, 'sale_amount': 57.0, 'sale_date': ...","{'item': 2, 'sale_amount': 55.0, 'sale_date': ...","{'item': 2, 'sale_amount': 66.0, 'sale_date': ...","{'item': 2, 'sale_amount': 79.0, 'sale_date': ...","{'item': 2, 'sale_amount': 60.0, 'sale_date': ...","

### 4. Save the data in your files to local csv files so that it will be faster to access in the future.

In [ ]:
df_items.to_csv("items_df.csv")
df_stores.to_csv("stores_df.csv")
df_sales.to_csv("sales_df.csv")

### 5. Combine the data from your three separate dataframes into one large dataframe.

In [ ]:
left = df_
  
right = df_
                          
# Merging the dataframes                      
df_ = pd.merge(left, right, how ='outer', on ='Sr.no') 

In [ ]:
left = df_
  
right = df_
                          
# Merging the dataframes                      
df_ = pd.merge(left, right, how ='outer', on ='Sr.no') 

### 6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [ ]:
url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'

c=pd.read_csv(url)

### 7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [ ]:
# Acquire.py Functions Using REST API that Returns JSON

import requests
import os
import numpy as np
import pandas as pd

######################################### Items DF ########################################

def get_items():
    
    if os.path.isfile('items_df.csv') == False:
        
        base_url = 'https://python.zach.lol/documentation'
        response = requests.get('https://python.zach.lol/api/v1/items')
        data = response.json()
        df_items = pd.DataFrame(data['payload']['items'])
        
    else:
        # Reads the csv saved from above, and assigns to the df variable
        df_items = pd.read_csv('items_df.csv', index_col=0)
        
    return df_items

######################################### Stores DF #######################################

def get_stores():
    
    if os.path.isfile('stores_df.csv') == False:
        
        response = requests.get('https://python.zach.lol/api/v1/stores')
        data = response.json()
        df_stores = pd.DataFrame(data['payload']['stores'])
        
    else:
        # Reads the csv saved from above, and assigns to the df variable
        df_stores = pd.read_csv('stores_df.csv', index_col=0)

    return df_stores
    
######################################### Sales DF ########################################

def get_sales():
    
    if os.path.isfile('sales_df.csv') == False:
        
        base_url = 'https://python.zach.lol/documentation'

        output = []
        for i in range(1, 184):
    
            response = requests.get(f"https://python.zach.lol/api/v1/sales?page={i}")
            data = response.json()
            output.append(data['payload']['sales'])
    
        df_sales = pd.DataFrame(output)
        
    else:
        # Reads the csv saved from above, and assigns to the df variable
        df_sales = pd.read_csv('sales_df.csv', index_col=0)
        
    return df_sales